# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [ ]:
from __future__ import print_function

import datetime
from tensorflow import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [ ]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [ ]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [ ]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()



11490434/11490434 [==============================] - 2s 0us/step


Create two datasets
* one with digits below 5
* one with 5 and above

In [ ]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [ ]:

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [ ]:

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [ ]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 7s 10ms/step - loss: 1.6108 - accuracy: 0.2177 - val_loss: 1.5811 - val_accuracy: 0.2615
Epoch 2/5
230/230 [==============================] - 2s 7ms/step - loss: 1.5755 - accuracy: 0.2780 - val_loss: 1.5422 - val_accuracy: 0.4090
Epoch 3/5
230/230 [==============================] - 2s 7ms/step - loss: 1.5390 - accuracy: 0.3412 - val_loss: 1.5016 - val_accuracy: 0.5390
Epoch 4/5
230/230 [==============================] - 2s 7ms/step - loss: 1.5012 - accuracy: 0.4075 - val_loss: 1.4572 - val_accuracy: 0.6219
Epoch 5/5
230/230 [==============================] - 2s 8ms/step - loss: 1.4599 - accuracy: 0.4651 - val_loss: 1.4082 - val_accuracy: 0.6754
Training time: 0:00:21.822436
Test score: 1.4081743955612183
Test accuracy: 0.6753754615783691


- The 2nd training has a greater training time but a higher accuracy

Freeze only the feature layers

In [ ]:

for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 3s 8ms/step - loss: 1.5650 - accuracy: 0.2783 - val_loss: 1.5102 - val_accuracy: 0.4799
Epoch 2/5
240/240 [==============================] - 1s 5ms/step - loss: 1.4986 - accuracy: 0.3989 - val_loss: 1.4430 - val_accuracy: 0.6180
Epoch 3/5
240/240 [==============================] - 2s 7ms/step - loss: 1.4402 - accuracy: 0.4930 - val_loss: 1.3786 - val_accuracy: 0.7015
Epoch 4/5
240/240 [==============================] - 1s 5ms/step - loss: 1.3798 - accuracy: 0.5753 - val_loss: 1.3156 - val_accuracy: 0.7519
Epoch 5/5
240/240 [==============================] - 1s 5ms/step - loss: 1.3227 - accuracy: 0.6336 - val_loss: 1.2534 - val_accuracy: 0.7896
Training time: 0:00:10.972100
Test score: 1.2534071207046509
Test accuracy: 0.7896478176116943


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [ ]:
epochs = 30

In [ ]:
new_model = Sequential(feature_layers + classification_layers)
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                      

In [ ]:
## Train on numbers 0-4
train_model(new_model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/30
240/240 [==============================] - 2s 6ms/step - loss: 0.3963 - accuracy: 0.9140 - val_loss: 0.3107 - val_accuracy: 0.9504
Epoch 2/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3894 - accuracy: 0.9143 - val_loss: 0.3052 - val_accuracy: 0.9510
Epoch 3/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3863 - accuracy: 0.9155 - val_loss: 0.2999 - val_accuracy: 0.9515
Epoch 4/30
240/240 [==============================] - 2s 7ms/step - loss: 0.3783 - accuracy: 0.9173 - val_loss: 0.2947 - val_accuracy: 0.9515
Epoch 5/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3733 - accuracy: 0.9163 - val_loss: 0.2898 - val_accuracy: 0.9519
Epoch 6/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3695 - accuracy: 0.9166 - val_loss: 0.2853 - val_accuracy: 0.9525
Epoch 7/30
240/240 [==============================] - 1s 5ms/step - loss: 0.

In [ ]:
for l in feature_layers:
    l.trainable = False

In [ ]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                      

In [ ]:
train_model(new_model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/30
230/230 [==============================] - 2s 5ms/step - loss: 1.9575 - accuracy: 0.3299 - val_loss: 1.8249 - val_accuracy: 0.3553
Epoch 2/30
230/230 [==============================] - 1s 6ms/step - loss: 1.8526 - accuracy: 0.3408 - val_loss: 1.7279 - val_accuracy: 0.3709
Epoch 3/30
230/230 [==============================] - 1s 6ms/step - loss: 1.7669 - accuracy: 0.3517 - val_loss: 1.6419 - val_accuracy: 0.3808
Epoch 4/30
230/230 [==============================] - 1s 5ms/step - loss: 1.6870 - accuracy: 0.3628 - val_loss: 1.5638 - val_accuracy: 0.3958
Epoch 5/30
230/230 [==============================] - 1s 5ms/step - loss: 1.6109 - accuracy: 0.3783 - val_loss: 1.4913 - val_accuracy: 0.4112
Epoch 6/30
230/230 [==============================] - 1s 5ms/step - loss: 1.5483 - accuracy: 0.3901 - val_loss: 1.4233 - val_accuracy: 0.4271
Epoch 7/30
230/230 [==============================] - 1s 5ms/step - loss: 1.

- The model trained 4 secs less than the first training

#### Conclusion

***During the activity I was able to get the gist of how transfer learning works, we train a model on a set of samples, then from the trainied model we than train it again on another set of samples and fine-tune it. On the First part of the model training, the model was trained a small amount of time but yielded a small accuracy; then it was trained agained on another set of samples and produced a higher accuracy but a longer training time. The reverse model had a different result, the first training had a higher accuracy and a bigger training time, the second training had a smaller training time but produced a lower accuracy than the first training.***